<a href="https://colab.research.google.com/github/Kingsley-Yoimiya/cuda-learning/blob/main/simple_practice/CUDA_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# add nvcc support for jupyter
!pip install nvcc4jupyter

%load_ext nvcc4jupyter

!nvcc --version

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpuw4_udth".
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
%%cuda

#include <iostream>
#include <random>
#include <chrono>
#include <cuda_runtime.h>
#include <iostream>
#include <ctime>
#include <cuda.h>

using namespace std;

#define BLOCK_SIZE 256

__global__ void reduce_sum_kernel(const float* input_vecs, size_t n, size_t dim, float* output_vec) {
    size_t idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx < n * dim) {
        atomicAdd(&output_vec[idx % dim], input_vecs[idx]);
    }
}

void reduce_sum(const float* input_vecs, size_t n, size_t dim, float* output_vec) {
    float* cu_input_vecs;
    float* cu_output_vecs;
    size_t input_size = n * dim * sizeof(float), output_size = dim * sizeof(float);
    cudaMalloc((void**) &cu_input_vecs, input_size);
    cudaMalloc((void**) &cu_output_vecs, output_size);
    cudaMemcpy(cu_input_vecs, input_vecs, input_size, cudaMemcpyHostToDevice);
    cudaMemset(cu_output_vecs, 0, output_size);
    size_t grid_size = (n * dim + BLOCK_SIZE - 1) / BLOCK_SIZE;
    reduce_sum_kernel <<< grid_size, BLOCK_SIZE >>>(cu_input_vecs, n, dim, cu_output_vecs);
    cudaDeviceSynchronize();
    cudaMemcpy(output_vec, cu_output_vecs, output_size, cudaMemcpyDeviceToHost);
    cudaFree(cu_input_vecs);
    cudaFree(cu_output_vecs);
}

const long long N = 1e9;
const int T = 1000;

uniform_real_distribution<float> u(0, 1);
mt19937 rnd(chrono::system_clock::now().time_since_epoch().count());

int main() {
    float* input_vecs = new float[N];
    float* output_vec = new float[T];
    float* correct_vec = new float[T];
    for(int i = 0; i < N; i++) {
        input_vecs[i] = u(rnd);
    }

    cerr << "GENERATE OK!" << endl;
    double st = clock();
    reduce_sum(input_vecs, N / T, T, output_vec);
    double ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;

    st = clock();

    for(int i = 0; i < T; i++) {
        correct_vec[i] = 0;
    }
    for(int i = 0; i < N; i++) {
        correct_vec[i % T] += input_vecs[i];
    }

    ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;

    for(int i = 0; i < T; i++) {
        if(abs(correct_vec[i] - output_vec[i]) > 1) {
            std::cout << correct_vec[i] << " " << output_vec[i] << " ERROR!" << std::endl;
            break;
        }
    }

    std::cout << output_vec[0] << std::endl;
    delete[] input_vecs;
    delete[] output_vec;

    return 0;
}


GENERATE OK!
0.991417
3.25612
500030



In [2]:
%%cuda

#include <iostream>
#include <random>
#include <chrono>
#include <cuda_runtime.h>
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <assert.h>

using namespace std;

#define GRID_SIZE 128
#define BLOCK_SIZE 512
#define cudaCheckError() {                                                      \
    cudaError_t err = cudaGetLastError();                                       \
    if (err != cudaSuccess) {                                                   \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)                  \
                  << " at " << __FILE__ << ":" << __LINE__ << std::endl;        \
        exit(EXIT_FAILURE);                                                     \
    }                                                                           \
}
#define cudaCheckErrorSync() {                                                  \
    cudaDeviceSynchronize();                                                    \
    cudaError_t err = cudaGetLastError();                                       \
    if (err != cudaSuccess) {                                                   \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)                  \
                  << " at " << __FILE__ << ":" << __LINE__ << std::endl;        \
        exit(EXIT_FAILURE);                                                     \
    }                                                                           \
}

#define cudaCheckErrorSync() {}
#define cudaCheckError() {}

__global__ void matmul_kernel(const float* A, const float* B, size_t n, size_t m, size_t k, float* output) {
    size_t idi = blockIdx.x * blockDim.x + threadIdx.x;
    size_t idj = blockIdx.y * blockDim.y + threadIdx.y;
    if(idi < n && idj < k) {
        float res = 0;
        for(int idk = 0; idk < m; idk++) {
            res += A[idi * m + idk] * B[idk * k + idj];
        }
        output[idi * k + idj] = res;
    }
}

void matmul(const float* A, const float* B, size_t n, size_t m, size_t k, float* output) {
    float* cu_A;
    float* cu_B;
    float* cu_output;
    size_t A_size = n * m * sizeof(float), B_size = m * k * sizeof(float), out_size = n * k * sizeof(float);
    cudaMalloc((void**) &cu_A, A_size);
    cudaCheckError();
    cudaMalloc((void**) &cu_B, B_size);
    cudaMalloc((void**) &cu_output, out_size);
    cudaCheckError();
    cudaMemcpy(cu_A, A, A_size, cudaMemcpyHostToDevice);
    cudaMemcpy(cu_B, B, B_size, cudaMemcpyHostToDevice);
    cudaCheckError();
    cudaMemset(cu_output, 0, out_size);
    dim3 grid(GRID_SIZE, GRID_SIZE);
    dim3 block((n + GRID_SIZE - 1) / GRID_SIZE, (k + GRID_SIZE - 1) / GRID_SIZE);
    cerr << (n + GRID_SIZE - 1) / GRID_SIZE << " " << (k + GRID_SIZE - 1) / GRID_SIZE << endl;
    matmul_kernel <<< grid, block >>>(cu_A, cu_B, n, m, k, cu_output);
    cudaCheckErrorSync();

    cudaDeviceSynchronize();
    cudaMemcpy(output, cu_output, out_size, cudaMemcpyDeviceToHost);
    cudaCheckError();
    cudaFree(cu_A);
    cudaFree(cu_B);
    cudaFree(cu_output);
}

const int N = 4096, M = 4096, K = 4096;
const int T = 100;

uniform_real_distribution<float> u(0, 1);
mt19937 rnd(chrono::system_clock::now().time_since_epoch().count());

int main() {
    float* A = new float[N * M];
    float* B = new float[M * K];
    float* C = new float[N * K];
    float* D = new float[N * K];
    for(int i = 0; i < N * M; i++) {
        A[i] = u(rnd);
    }
    for(int i = 0; i < M * K; i++) {
        B[i] = u(rnd);
    }
    cerr << "GENERATE OK!" << endl;

    double st = clock();
    matmul(A, B, N, M, K, C);
    double ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;

    st = clock();
    for(int i = 0; i < N * K; i++) {
        D[i] = 0;
    }
    for(int i = 0; i < N; i++) {
       for(int j = 0; j < M; j++) {
            for(int k = 0; k < K; k++) {
                D[i * K + k] += A[i * M + j] * B[j * K + k];
            }
       }
    }

    ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;

    for(int i = 0; i < N * K; i++) {
        if(fabs(C[i] - D[i]) > 1e-2) {
            std::cout << C[i] << " " << D[i] << " ERROR!" << std::endl;
            break;
        }
    }

    std::cout << "RESULT: " << C[0] << std::endl;
    delete[] A;
    delete[] B;
    delete[] C;
    delete[] D;
    return 0;
}

GENERATE OK!
32 32
2.62281
297.349
RESULT: 1023.66



In [7]:
%%cuda

#include <iostream>
#include <random>
#include <chrono>
#include <cuda_runtime.h>
#include <iostream>
#include <ctime>
#include <cuda.h>
#include <assert.h>

using namespace std;

#define GRID_SIZE 128
#define BLOCK_SIZE 256
#define BATCH 131072
#define cudaCheckError() {                                                      \
    cudaError_t err = cudaGetLastError();                                       \
    if (err != cudaSuccess) {                                                   \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)                  \
                  << " at " << __FILE__ << ":" << __LINE__ << std::endl;        \
        exit(EXIT_FAILURE);                                                     \
    }                                                                           \
}
#define cudaCheckErrorSync() {                                                  \
    cudaDeviceSynchronize();                                                    \
    cudaError_t err = cudaGetLastError();                                       \
    if (err != cudaSuccess) {                                                   \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)                  \
                  << " at " << __FILE__ << ":" << __LINE__ << std::endl;        \
        exit(EXIT_FAILURE);                                                     \
    }                                                                           \
}

// #define cudaCheckErrorSync() {}
// #define cudaCheckError() {}

int nextPow2(int x) {
    x--;
    for(int i = 1; i < 32; i <<= 1) x |= x >> i;
    return x + 1;
}

__global__ void debubble_kernel_1(const int *a, int n, int *blockpre, int *store) {
    __shared__ int s[BLOCK_SIZE];
    int st = (blockIdx.x * blockDim.x + threadIdx.x) * BATCH, ed = st + BATCH;
    int blockid = blockIdx.x;
    int cnt = 0;
    for(int i = min(st, n); i < min(ed, n); i++) cnt += a[i] != 0;
    s[threadIdx.x] = store[blockIdx.x * blockDim.x + threadIdx.x] = cnt;
    __syncthreads();
    for(int i = 1; i < blockDim.x; i <<= 1) {
        if(threadIdx.x + i < blockDim.x) s[threadIdx.x + i] += s[threadIdx.x];
        __syncthreads();
    }
    if(threadIdx.x == 0) blockpre[blockid] = s[blockDim.x - 1];
}

__global__ void debubble_kernel_2(const int *a, int n, const int *blockpre, const int *store, int *output) {
    __shared__ int s[BLOCK_SIZE];
    int st = (blockIdx.x * blockDim.x + threadIdx.x) * BATCH, ed = st + BATCH;
    int cnt = store[blockIdx.x * blockDim.x + threadIdx.x];
    //for(int i = min(st, n); i < min(ed, n); i++) cnt += a[i] != 0;
    s[threadIdx.x] = cnt;
    __syncthreads();
    for(int i = 1, totnum = blockDim.x >> 1; i < blockDim.x; i <<= 1, totnum >>= 1) {
        if(threadIdx.x < totnum) {
            int id1 = threadIdx.x * (i << 1) + i - 1, id2 = id1 + i;
            s[id2] += s[id1];
        }
        __syncthreads();
    }
    if(threadIdx.x == 0) s[blockDim.x - 1] = 0;
    for(int i = blockDim.x >> 1, totnum = 1; i > 0; i >>= 1, totnum <<= 1) {
        if(threadIdx.x < totnum) {
            int id1 = threadIdx.x * (i << 1) + i - 1, id2 = id1 + i;
            int tmp = s[id1];
            s[id1] = s[id2];
            s[id2] += tmp;
        }
        __syncthreads();
    }

    cnt = s[threadIdx.x] + blockpre[blockIdx.x];
    for(int i = min(st, n); i < min(ed, n); i++) {
        if(a[i] != 0) {
            output[cnt] = a[i];
            cnt++;
        }
    }
}

__global__ void preSum(int *a, int tot, size_t *ans) {
    __shared__ int s[BLOCK_SIZE];
    s[threadIdx.x] = a[threadIdx.x];
    __syncthreads();
    for(int i = 1, totnum = blockDim.x >> 1; i < blockDim.x; i <<= 1, totnum >>= 1) {
        if(threadIdx.x < totnum) {
            int id1 = threadIdx.x * (i << 1) + i - 1, id2 = id1 + i;
            s[id2] += s[id1];
        }
        __syncthreads();
    }
    if(threadIdx.x == 0) ans[0] = s[blockDim.x - 1], s[blockDim.x - 1] = 0;
    for(int i = blockDim.x >> 1, totnum = 1; i > 0; i >>= 1, totnum <<= 1) {
        if(threadIdx.x < totnum) {
            int id1 = threadIdx.x * (i << 1) + i - 1, id2 = id1 + i;
            int tmp = s[id1];
            s[id1] = s[id2];
            s[id2] += tmp;
        }
        __syncthreads();
    }
    a[threadIdx.x] = s[threadIdx.x];
}

size_t debubble(vector < int >&a) {
    int* cu_a;
    int* blockpre;
    int* output;
    int *store;
    int n = a.size(), totcore = (n + BATCH - 1) / BATCH, blocknum = nextPow2((totcore + BLOCK_SIZE - 1) / BLOCK_SIZE);
    cout << totcore << " " << blocknum << endl;
    cudaMalloc((void**) &cu_a, n * sizeof(int));
    cudaCheckError();
    cudaMalloc((void**) &blockpre, (blocknum) * sizeof(int));
    cudaMalloc((void**) &output, n * sizeof(int));
    cudaMalloc((void**) &store , blocknum * BLOCK_SIZE * sizeof(int));
    cudaMemcpy(cu_a, a.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaCheckError();
    cudaMemset(blockpre, 0, (blocknum)  * sizeof(int));
    cudaMemset(output, 0, n * sizeof(int));
    cudaMemset(store, 0, blocknum * BLOCK_SIZE * sizeof(int));
    cudaCheckError();
    // cout << "!" << endl;
    debubble_kernel_1 <<< blocknum, BLOCK_SIZE >>>(cu_a, n, blockpre, store);
    cudaDeviceSynchronize();
    cudaCheckError();
    // cout << "!1" << " " << blocknum << endl;

    // illigal:
    // for(int i = 1; i < blocknum; i++) cout << i << endl, blockpre[i] += blockpre[i - 1];
    size_t * cu_ans, ans;
    cudaMalloc((void**) &cu_ans, sizeof(size_t));
    preSum <<< 1, blocknum >>> (blockpre, blocknum, cu_ans);
    cudaMemcpy(&ans, cu_ans, sizeof(size_t), cudaMemcpyDeviceToHost);
    // cout << "!2" << " " << ans << endl;
    cudaCheckError();
    debubble_kernel_2 <<< blocknum, BLOCK_SIZE >>>(cu_a, n, blockpre, store, output);
    cudaCheckError();
    cudaDeviceSynchronize();
    // cout << "!3" << endl;
    cudaCheckError();
    // cout << "!" << endl;
    cudaMemcpy(a.data(), output, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaCheckError();
    // cout << "!" << endl;
    cudaFree(cu_a);
    cudaFree(blockpre);
    cudaFree(output);
    cudaCheckError();
    return ans;
}

const int N = 536870912, M = 2000, K = 2000;
const int T = 100;

uniform_real_distribution<float> u(0, 1);
mt19937 rnd(chrono::system_clock::now().time_since_epoch().count());

int main() {
    vector < int > a, b, c;
    for(int i = 0; i < N; i++) {
        a.emplace_back(rnd() % 1024);
    }
    b = a;
    cerr << "GENERATE OK!" << a.size() << endl;

    double st = clock();
    cout << debubble(a) << endl;
    double ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;

    // for(int i = 0; i < 10; i++) cout << a[i] << endl;

    st = clock();
    for(int x : b) if(x != 0) c.emplace_back(x);
    while(c.size() < b.size()) c.emplace_back(0);

    ed = clock();
    std::cout << (ed - st) / CLOCKS_PER_SEC << std::endl;
    for(int i = 0; i < N; i++) if(a[i] != c[i]) {
        cout << i << " " << a[i] << " " << c[i] << endl;
        break;
    }

    cout << a.size() << " " << c.size() << endl;
    assert(a == c);
    return 0;
}
/*GENERATE OK!536870912
8192 32
536347455
1.34073
28.7449
536870912 536870912
*/

GENERATE OK!536870912
4096 16
536345938
1.44263
28.5526
536870912 536870912

